In [1]:
from sgnlp.models.lsr import LsrModel, LsrConfig, LsrPreprocessor, LsrPostprocessor
from transformers import cached_path
from text_input_to_docred_pipeline import TextInputToDocredPipeline 

In [2]:
# Download files from azure blob storage
rel2id_path = cached_path('https://storage.googleapis.com/sgnlp/models/lsr/rel2id.json')
word2id_path = cached_path('https://storage.googleapis.com/sgnlp/models/lsr/word2id.json')
ner2id_path = cached_path('https://storage.googleapis.com/sgnlp/models/lsr/ner2id.json')
rel_info_path = cached_path('https://storage.googleapis.com/sgnlp/models/lsr/rel_info.json')

In [3]:
PRED_THRESHOLD = 0.3
# text2docred_pipeline = TextInputToDocredPipeline()
preprocessor = LsrPreprocessor(rel2id_path=rel2id_path, word2id_path=word2id_path, ner2id_path=ner2id_path)
postprocessor = LsrPostprocessor.from_file_paths(rel2id_path=rel2id_path, rel_info_path=rel_info_path,
                                                 pred_threshold=PRED_THRESHOLD)


In [4]:
text2docred_pipeline = TextInputToDocredPipeline()

/opt/anaconda3/envs/AISG/lib/python3.8/site-packages/allennlp/data/token_indexers/token_characters_indexer.py:55: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  warnings.warn(
/opt/anaconda3/envs/AISG/lib/python3.8/site-packages/torch/nn/modules/container.py:435: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")
Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
text_content='Student Experience Survey 2023 - President Message'
docred_doc = text2docred_pipeline.preprocess(text_content)

In [13]:
# can exstract valuable information in text but not so accurate
docred_doc['vertexSet']

[]

In [6]:
config = LsrConfig.from_pretrained('https://storage.googleapis.com/sgnlp/models/lsr/v2/config.json')
model = LsrModel.from_pretrained('https://storage.googleapis.com/sgnlp/models/lsr/v2/pytorch_model.bin', config=config)
model.eval()

LsrModel(
  (dropout): Dropout(p=0.3, inplace=False)
  (relu): ReLU()
  (word_emb): Embedding(194784, 100)
  (ner_emb): Embedding(13, 20, padding_idx=0)
  (coref_embed): Embedding(512, 20, padding_idx=0)
  (linear_re): Linear(in_features=240, out_features=120, bias=True)
  (rnn_sent): Encoder(
    (encoder): GRU(140, 120, bidirectional=True)
    (dropout_embedding): Dropout(p=0.2, inplace=False)
    (dropout_encoder): Dropout(p=0.3, inplace=False)
  )
  (dropout_gcn): Dropout(p=0.4, inplace=False)
  (reasoner): ModuleList(
    (0): DynamicReasoner(
      (dropout_gcn): Dropout(p=0.4, inplace=False)
      (struc_att): StructInduction(
        (tp_linear): Linear(in_features=60, out_features=60, bias=True)
        (tc_linear): Linear(in_features=60, out_features=60, bias=True)
        (fi_linear): Linear(in_features=60, out_features=1, bias=False)
        (bilinear): Bilinear(in1_features=60, in2_features=60, out_features=1, bias=False)
        (fzlinear): Linear(in_features=180, out_fea

In [58]:
# if len(docred_doc["vertexSet"]) != 0:
# not work if the relationship is not strong
tensor_doc = preprocessor([docred_doc])
output = model(**tensor_doc)
result = postprocessor(output.prediction, [docred_doc])[0]['relations']
result

RuntimeError: shape '[1, 1, 140]' is invalid for input of size 0